In [1]:
import pyaudio
import wave
import threading
import ipywidgets as widgets
import random
import os
import time

start_button = dict()
stop_button = dict()

# Set the parameters for the recording
CHUNK = 1024 # Number of samples per frame
FORMAT = pyaudio.paInt16 # Sample format
CHANNELS = 1 # Mono recording
RATE = 44100 # Sample rate (Hz)

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Create a flag to signal when recording should stop
stop_recording = False
languages = ("ENGLISH", "TULU") 

randNum = random.randint(0, 99999999)

# Define a function to record audio
def record_audio(randNum, language):
    global stop_recording
    # Open a new stream to record audio
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    # Create a list to store the recorded frames
    frames = []

    # Record audio until the stop_recording flag is set
    while not stop_recording:
        # Read a frame of audio data from the stream
        data = stream.read(CHUNK)
        # Add the frame to the list of recorded frames
        frames.append(data)

    # Stop and close the stream when recording is finished
    stream.stop_stream()
    stream.close()

    # Terminate PyAudio
#     audio.terminate()

    # Save the recorded audio as a WAV file
    if (not os.path.exists(language)):
        os.mkdir(language)
    with wave.open(f"{language}/{randNum}.wav", "wb") as wav_file:
        wav_file.setnchannels(CHANNELS)
        wav_file.setsampwidth(audio.get_sample_size(FORMAT))
        wav_file.setframerate(RATE)
        wav_file.writeframes(b"".join(frames))
        
# Define a function to start the recording
def start_recording(button, randNum):
    global stop_recording
    # Disable the start button
    language = "ENGLISH" if "ENGLISH" in button.description else "TULU"
    start_button[language].disabled = True
    # Enable the stop button
    stop_button[language].disabled = False
    # Clear the output area
    output_area.clear_output()
    # Reset the stop_recording flag
    stop_recording = False
    # Start a new thread to record audio
    threading.Thread(target=record_audio, args=(randNum, language)).start()

# Define a function to stop the recording
def stop_rec(button):
    global stop_recording
    language = "ENGLISH" if "ENGLISH" in button.description else "TULU"
    # Set the stop_recording flag
    stop_recording = True
    # Disable the stop button
    stop_button[language].disabled = True
    # Enable the start button
    start_button[language].disabled = False
    
    
output_area = widgets.Output()

input_text = widgets.Text()
display(input_text, output_area)

for language in languages:
    start_button[language] = widgets.Button(description=f"Start Recording {language}")
    start_button[language].on_click(lambda b: start_recording(b, randNum))
    
    stop_button[language] = widgets.Button(description=f"Stop Recording {language}", disabled=True)
    stop_button[language].on_click(lambda b: stop_rec(b))

    # Create an output area to display messages

    # Display the buttons and output area
    display(widgets.HBox([start_button[language], stop_button[language]]), output_area)

def Next_funtion(button):
    if input_text.value != '' and os.path.exists(f"ENGLISH/{randNum}.wav") and os.path.exists(f"TULU/{randNum}.wav"):
        with open("data.csv", "a+") as f:
              f.write(f"{randNum},{input_text.value}")
        button.button_style = "success"
        time.sleep(0.5)
        button.button_style = ""
    else:
        button.button_style = "danger"
        time.sleep(0.5)
        button.button_style = ""

def Cancel_funtion(button):
    if os.path.exists(f"ENGLISH/{randNum}.wav"):
        os.remove(f"ENGLISH/{randNum}.wav")
        
    if os.path.exists(f"TULU/{randNum}.wav"):
        os.remove(f"TULU/{randNum}.wav")
        
    button.button_style = "danger"
    time.sleep(0.5)
    button.button_style = ""
    
        
NEXT = widgets.Button(description="NEXT")
NEXT.on_click(Next_funtion)
Cancel = widgets.Button(description="Cancel")
Cancel.on_click(Cancel_funtion)
display(widgets.HBox([NEXT, Cancel]), output_area)

Text(value='')

Output()

Output()

Output()

Output()